 Useful links:
 - [Flax_basics](https://flax.readthedocs.io/en/latest/guides/flax_basics.html)
 - [Training-Loop-in-JAX](https://wandb.ai/jax-series/simple-training-loop/reports/Writing-a-Training-Loop-in-JAX-FLAX--VmlldzoyMzA4ODEy)
 - [Kaggle example](https://www.kaggle.com/code/nilaychauhan/digit-recognizer-using-jax-flax/notebook)

In [ ]:
# %env JAX_ENABLE_X64=1
%env JAX_PLATFORM_NAME=cpu
# %env JAX_DISABLE_JIT=1
# %env JAX_DEBUG_NANS=1

In [ ]:
import jax
from jax import random, lax, jit
from flax import linen as nn
from typing import Sequence, Callable
import optax
from flax.training import train_state
import jax.numpy as jnp
from collections import defaultdict
import matplotlib.pylab as plt
import numpy as np

In [ ]:
class Net(nn.Module):
    features: Sequence[int]

    def setup(self):
        self.layers = [nn.Dense(feat) for feat in self.features]

    def __call__(self, inputs):
        x = inputs
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i != len(self.layers) - 1:
                x = nn.tanh(x)
        return x

In [ ]:
key1, key2 = random.split(random.PRNGKey(0), 2)

def generate_x(shape):
    return 5 * jnp.pi * jnp.asarray(np.random.uniform(size=shape)) # random.uniform(key1, shape)

def target(x):
    return jnp.cos(x)

x = generate_x((512, 1))
# x

In [ ]:
model = Net(features=[512, 256, 128, 64, 32, 1])
params = model.init(key2, x)
jax.tree_map(lambda x: x.shape, params) # Check the parameters

In [ ]:
y = model.apply(params, x)
y.shape

In [ ]:
def init_train_state(
    model, 
    random_key, 
    shape, 
    learning_rate,
) -> train_state.TrainState:
    
    # Initialize the Model
    variables = model.init(random_key, jnp.ones(shape))
    
    # Create the optimizer
    optimizer = optax.adam(learning_rate)
    # optimizer = optax.sgd(0.0001, 0.9)
    
    # Create a State
    return train_state.TrainState.create(
        apply_fn = model.apply,
        tx=optimizer,
        params=variables['params'],
    )

state = init_train_state(
    model, key2, (16, 1), 0.001
)
# state

In [ ]:
@jit
def train_step(
    state: train_state.TrainState, 
    batch: jnp.ndarray,
):
    x, y = batch

    def loss_fn(params):
        logits = state.apply_fn({'params': params}, x)
        loss = jnp.mean((logits - y)**2) # MSE, compute_loss
        return loss, logits

    gradient_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (_, logits), grads = gradient_fn(state.params)
    state = state.apply_gradients(grads=grads)
    metrics = jnp.mean((logits - y)**2) # MSE, compute_metrics
    return state, metrics

@jit
def eval_step(
    state: train_state.TrainState, 
    batch: jnp.ndarray,
):
    x, y = batch
    logits = state.apply_fn({'params': state.params}, x)
    return jnp.mean((logits - y)**2)

In [ ]:
history = defaultdict(list)

for epoch in range(1000):
    x_ = generate_x((16, 1))
    batch = x_, target(x_) 
    state, metrics = train_step(state, batch)
    if epoch % 10 == 0:
        print(f"{epoch=}, metric={metrics}")
        # print(state.params['layers_0']['bias'])
    history['epoch'].append(epoch)
    history['metrics_train'].append(metrics)

In [ ]:
plt.plot(history['epoch'], history['metrics_train'])
plt.xlabel("epoch")
plt.ylabel("metrics");

In [ ]:
x = generate_x((512, 1))
y_pred = model.apply({'params': state.params}, x)
y_true = target(x)
print(jnp.mean((y_pred - y_true)**2))

plt.scatter(x, y_pred, label="neural network")
plt.scatter(x, y_true, label="target fucntion")
plt.legend()

In [ ]:
class SimpleDense(nn.Module):
    features: int
    kernel_init: Callable = nn.initializers.lecun_normal()
    bias_init: Callable = nn.initializers.zeros

    @nn.compact
    def __call__(self, inputs):
        kernel = self.param('kernel',
                            self.kernel_init, # Initialization function
                            (inputs.shape[-1], self.features))  # shape info.
        y = lax.dot_general(inputs, kernel,
                            (((inputs.ndim - 1,), (0,)), ((), ())),) # TODO Why not jnp.dot?
        bias = self.param('bias', self.bias_init, (self.features,))
        y = y + bias
        return y

key1, key2 = random.split(random.PRNGKey(0), 2)
x = random.uniform(key1, (4,4))

model = SimpleDense(features=3)
params = model.init(key2, x)
y = model.apply(params, x)

print('initialized parameters:\n', params)
print('output:\n', y)